In [2]:
import numpy as np
import pandas as pd
import random
import joblib as jl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import Normalizer,MinMaxScaler
from sklearn.linear_model import LinearRegression
import warnings

# Tắt cảnh báo tạm thời
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # Đoạn mã có thể gây ra cảnh báo
    # ...

# Sau khi khối mã kết thúc, cảnh báo sẽ hoạt động lại bình thường

# feature infomation
# plot correlation matrix->Choose 1 most important feature -> Simple Linear(Model Scratch,model sklearn)->model base->feature selection
# adjust params(gridsearch,default)
# K-folds, metrics

In [3]:
df=pd.read_csv(r"C:\Users\NTCUONG\notebook\house_training.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
df=df.drop(columns=["Id","Alley","MasVnrType","FireplaceQu","PoolQC","Fence","MiscFeature"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [5]:
df.to_csv("data_process_ipt.csv",index=False)

In [6]:
df["Ageofhouse"]=df["YrSold"]-df["YearBuilt"]
sale_price=df["SalePrice"]
df=pd.concat([df.iloc[:,:-2],df["Ageofhouse"]],axis=1)
df=pd.concat([df,sale_price],axis=1)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Ageofhouse,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2,2008,WD,Normal,5,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,5,2007,WD,Normal,31,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,9,2008,WD,Normal,7,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,2,2006,WD,Abnorml,91,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,12,2008,WD,Normal,8,250000


In [7]:
columns_contain_nan=[]
for column in df.columns:
    if df[column].isna().sum()!=0:
        columns_contain_nan.append([column,(df[column].dtype)])
columns_contain_nan

[['LotFrontage', dtype('float64')],
 ['MasVnrArea', dtype('float64')],
 ['BsmtQual', dtype('O')],
 ['BsmtCond', dtype('O')],
 ['BsmtExposure', dtype('O')],
 ['BsmtFinType1', dtype('O')],
 ['BsmtFinType2', dtype('O')],
 ['Electrical', dtype('O')],
 ['GarageType', dtype('O')],
 ['GarageYrBlt', dtype('float64')],
 ['GarageFinish', dtype('O')],
 ['GarageQual', dtype('O')],
 ['GarageCond', dtype('O')]]

In [8]:
df_label_enc=df.copy()

In [9]:
enc=LabelEncoder()
df_num = df_label_enc.select_dtypes(include = ['object'])
obj_columns=df_num.columns
for column in obj_columns:
    enc.fit(df_label_enc[column])
    df_label_enc[column]=enc.transform(df_label_enc[column])

columns_contain_nan=[]
for column in df_label_enc.columns:
    if df_label_enc[column].isna().sum()!=0:
        columns_contain_nan.append([column,(df_label_enc[column].dtype)])
columns_contain_nan

[['LotFrontage', dtype('float64')],
 ['MasVnrArea', dtype('float64')],
 ['GarageYrBlt', dtype('float64')]]

In [10]:
df_label_enc.to_csv("data_label_enc.csv",index=False)

In [11]:
df_oh_enc=df.copy()
df_oh_enc=pd.get_dummies(df,dtype=int,dummy_na=True)
df_oh_enc

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,1,0,0,0,0,0,1,0,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,1,0,0,0,0,0,1,0,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,1,0,0,0,0,0,1,0,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,1,0,1,0,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,1,0,0,0,0,0,1,0,0
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,1,0,0,0,0,0,1,0,0
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,1,0,0,0,0,0,1,0,0
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,0,1,0,0,0,0,0,1,0,0


In [12]:
SPrice_idx=df_oh_enc.columns.get_loc("SalePrice")
print(SPrice_idx)
df_oh_enc=pd.concat([df_oh_enc.iloc[:,:SPrice_idx],df_oh_enc.iloc[:,SPrice_idx+1:],df_oh_enc.iloc[:,SPrice_idx]],axis=1)
df_oh_enc

37


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,1,0,0,0,0,0,1,0,0,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,1,0,0,0,0,0,1,0,0,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,1,0,0,0,0,0,1,0,0,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,1,0,1,0,0,0,0,0,0,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,1,0,0,0,0,0,1,0,0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,1,0,0,0,0,0,1,0,0,175000
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,1,0,0,0,0,0,1,0,0,210000
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,1,0,0,0,0,0,1,0,0,266500
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,1,0,0,0,0,0,1,0,0,142125


In [13]:
imputer_list={"simpleimputer":SimpleImputer(),"knnimputer":KNNImputer()}
norm_list={"none":"NONE","normalizer":Normalizer(),"minmaxscaler":MinMaxScaler()}
df_list={"LabelEncoder":df_label_enc,"OnehotEncoder":df_oh_enc}

In [14]:
names_columns_nan=[column[0] for column in columns_contain_nan]

In [15]:
names_columns_nan

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [16]:
def process(df,imputer,norm,names_columns_nan,i):
    ipt=imputer
    n=norm
    for column in names_columns_nan:
        df[column]=ipt.fit_transform(df[column].to_numpy().reshape(-1,1))
    if norm!="NONE":
        df.iloc[:,:-1]=n.fit_transform(df.iloc[:,:-1])
    return df
# names_columns_nan=[column[0] for column in columns_contain_nan]
# processed_df=process(df_label_enc,SimpleImputer(),Normalizer(),names_columns_nan)
# processed_df

In [17]:
i=1
training_stt={"DATASET":[],"ENCODER":[],"IMPUTER":[],"NORMALIZER":[]}
for enc,df in df_list.items():
    for ipt_name,ipt in imputer_list.items():
        for norm_name,norm in norm_list.items():
            data=process(df,ipt,norm,names_columns_nan,i)
            training_stt["DATASET"].append("training{}.csv".format(i))
            training_stt["IMPUTER"].append(ipt_name)
            training_stt["ENCODER"].append(enc)
            training_stt["NORMALIZER"].append(norm_name)
            training_df=pd.DataFrame(data,columns=df.columns)
            training_df.to_csv("training{}.csv".format(i),index=False)
            i+=1
stt_df=pd.DataFrame(training_stt)
stt_df.to_csv("statistic_training_dataset.csv",index=False)


C:\Users\NTCUONG\AppData\Local\Temp\ipykernel_2412\783775067.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.00620633 0.00187296 0.00491416 ... 0.00655396 0.00186655 0.00182184]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:,:-1]=n.fit_transform(df.iloc[:,:-1])
C:\Users\NTCUONG\AppData\Local\Temp\ipykernel_2412\783775067.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.00031032 0.00028094 0.00024571 ... 0.00028088 0.00027998 0.00027328]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.iloc[:,:-1]=n.fit_transform(df.iloc[:,:-1])
C:\Users\NTCUONG\AppData\Local\Temp\ipykernel_2412\783775067.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.874058   0.89901912 0.92

In [18]:
df_oh_enc

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan,SalePrice
0,0.288964,0.288900,0.237060,0.751485,0.490417,0.871241,0.865572,0.127085,0.386851,0.000000,...,0.878061,0,0.000000,0.0,0.0,0.0,0.878061,0.0,0,208500
1,0.089223,0.343729,0.262077,0.610590,0.764289,0.820615,0.815275,0.000000,0.514486,0.000000,...,0.842986,0,0.000000,0.0,0.0,0.0,0.842986,0.0,0,181500
2,0.277911,0.287757,0.312420,0.717292,0.467697,0.830206,0.825241,0.101427,0.257144,0.000000,...,0.847863,0,0.000000,0.0,0.0,0.0,0.847863,0.0,0,223500
3,0.323125,0.252112,0.260167,0.716555,0.467473,0.792212,0.810866,0.000000,0.113340,0.000000,...,0.840844,0,0.846186,0.0,0.0,0.0,0.000000,0.0,0,140000
4,0.249337,0.320285,0.360368,0.735365,0.415464,0.737297,0.732499,0.197518,0.312378,0.000000,...,0.764231,0,0.000000,0.0,0.0,0.0,0.764231,0.0,0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.288205,0.274667,0.219535,0.638845,0.489865,0.868407,0.863207,0.000000,0.000000,0.000000,...,0.875128,0,0.000000,0.0,0.0,0.0,0.875128,0.0,0,175000
1456,0.082721,0.339312,0.344205,0.565160,0.527383,0.761718,0.760856,0.069883,0.392058,0.114365,...,0.795260,0,0.000000,0.0,0.0,0.0,0.795260,0.0,0,210000
1457,0.316322,0.273387,0.234047,0.701315,0.842639,0.786429,0.808859,0.000000,0.141273,0.000000,...,0.823215,0,0.000000,0.0,0.0,0.0,0.823215,0.0,0,266500
1458,0.092958,0.301147,0.277546,0.524039,0.590862,0.843130,0.858447,0.000000,0.026861,0.797479,...,0.878429,0,0.000000,0.0,0.0,0.0,0.878429,0.0,0,142125
